In [1]:
import sqlite3
import pandas as pd

# sqlitebrowser

# create diagram https://dbdiagram.io/d

# Create sql from csv

In [2]:
genstudio = pd.read_csv('./data/genstudio.csv', dtype={'Position':str})
metadata = pd.read_csv('./data/metadata.csv')

#print(genstudio.shape, metadata.shape)

In [14]:
col_names, types = genstudio.columns, genstudio.dtypes
scheme_genstudio = ['row_id INTEGER PRIMARY KEY']
new_names = ['row_id']


for name, contain in zip(col_names[1:], types[1:]):
    name = name.replace("-", "").replace("  ", " ").replace(" ", "_")
    new_names.append(name)

    if 'int' in str(contain):
        scheme_genstudio.append(' '.join([name, 'INTEGER']))
    
    elif 'float' in str(contain):
        scheme_genstudio.append(' '.join([name, 'REAL']))
    
    else:
        scheme_genstudio.append(' '.join([name, 'TEXT']))
        
scheme_genstudio = ', '.join(scheme_genstudio)
genstudio.columns = new_names

In [16]:
col_names, types = metadata.columns, metadata.dtypes
scheme_metadata = ['row_id INTEGER PRIMARY KEY']
new_names = ['row_id']

for name, contain in zip(col_names[1:], types[1:]):
    name = name.replace("-", "").replace("  ", " ").replace(" ", "_")
    new_names.append(name)

    if 'int' in str(contain):
        scheme_metadata.append(' '.join([name, 'INTEGER']))
    
    if 'float' in str(contain):
        scheme_metadata.append(' '.join([name, 'REAL']))
    
    else:
        scheme_metadata.append(' '.join([name, 'TEXT']))
        
scheme_metadata = ', '.join(scheme_metadata)
metadata.columns = new_names

In [78]:
connection = sqlite3.connect('./data/bid_DB.db')
#c_genstudio = conn_genstudio.cursor()

connection.execute(f'''CREATE TABLE IF NOT EXISTS SNP_data (
Sample_ID TEXT, 
SNP_Name TEXT, 
SNP TEXT, 
Position TEXT, 
Chr TEXT,
sex TEXT)''')

connection.commit()

connection.execute(f'''CREATE TABLE IF NOT EXISTS metadata ({scheme_metadata},
                       FOREIGN KEY (sex) REFERENCES SNP_data (sex))''')
connection.commit()

connection.execute(f'''CREATE TABLE IF NOT EXISTS genstudio ({scheme_genstudio},
FOREIGN KEY (Sample_ID) REFERENCES metadata (dna_chip_id),
FOREIGN KEY (Sample_ID) REFERENCES metadata (Sample_ID),
FOREIGN KEY (SNP_Name) REFERENCES metadata (SNP_Name),
FOREIGN KEY (SNP) REFERENCES metadata (SNP),
FOREIGN KEY (Position) REFERENCES metadata (Position),
FOREIGN KEY (Chr) REFERENCES metadata (Chr))''')

connection.commit()

In [79]:
genstudio.to_sql('genstudio', connection, if_exists='append', index=False)
metadata.to_sql('metadata', connection, if_exists='append', index=False)

connection.close()

# Selection

In [80]:
connection = sqlite3.connect('./data/bid_DB.db')

query = '''SELECT metadata.dna_chip_id, genstudio.SNP_Name, genstudio.SNP, genstudio.Position, genstudio.Chr, 
           metadata.sex
           FROM genstudio, metadata
           WHERE metadata.dna_chip_id = genstudio.Sample_ID
'''

res = connection.execute(query)

print(len(res.fetchall()))

connection.close()

## Add selection to DB

In [82]:
connection = sqlite3.connect('./data/bid_DB.db')

query = '''INSERT INTO SNP_data (Sample_ID, SNP_Name, SNP, Position, Chr, sex)
SELECT genstudio.Sample_ID, genstudio.SNP_Name, genstudio.SNP, genstudio.Position, genstudio.Chr, 
metadata.sex 
FROM genstudio, metadata 
WHERE metadata.dna_chip_id = genstudio.Sample_ID'''

connection.execute(query)
connection.commit()
connection.close()

# Create and add from web

In [ ]:
connection = sqlite3.connect('./data/SNP_human.db')

connection.execute(f'''CREATE TABLE IF NOT EXISTS SNP_data (
Sample_ID TEXT, 
SNP_Name TEXT, 
SNP TEXT, 
Position TEXT, 
Chr TEXT,
sex TEXT)''')